# Cross Dataset Analysis

In [9]:
dataset_id = "reporting"

In [10]:
!bq --location=US mk --dataset {dataset_id}

Dataset 'sashimi-266523:reporting' successfully created.


## Number of public resource establishments vs income

In [7]:
%%bigquery
select ZIPCODE, Average as average_income, count(ZIPCODE) as num_of_public_resources
from `acs_2018_modeled.Income_Beam_DF` i 
left join `uds_mapper_modeled.ZIP_To_ZCTA5_Beam_DF` m using(ZCTA5)
left join `iml_2018_modeled.Public_Resources_Beam_DF` p using(ZIPCODE)
group by ZIPCODE, Average
order by num_of_public_resources desc

,ZIPCODE,average_income,num_of_public_resources
0,74743,18.8560,10
1,95616,32.9756,9
2,85287,20.2002,9
3,48202,16.8258,8
4,31401,21.6684,8
...,...,...,...
6243,81125,21.9658,1
6244,65793,21.1976,1
6245,70514,24.2302,1
6246,30083,20.6782,1


#### create view

In [13]:
%%bigquery
create or replace view reporting.v_Public_Resources_with_Income as
select ZIPCODE, Average as average_income, count(ZIPCODE) as num_of_public_resources
from `sashimi-266523.acs_2018_modeled.Income_Beam_DF` i 
left join `sashimi-266523.uds_mapper_modeled.ZIP_To_ZCTA5_Beam_DF` m using(ZCTA5)
left join `sashimi-266523.iml_2018_modeled.Public_Resources_Beam_DF` p using(ZIPCODE)
group by ZIPCODE, Average
order by num_of_public_resources desc

""


### Number of public school and private school in areas in comparison to income of the area/neighborhood

In [9]:
%%bigquery
select i.ZCTA5, Predominant_Class, number_of_private_schools, number_of_public_schools
from (acs_2018_modeled.Income_Beam_DF i join uds_mapper_modeled.ZIP_To_ZCTA5_Beam_DF z on i.ZCTA5 = z.ZCTA5) join 
(select pri.ZIPCODE, number_of_private_schools, number_of_public_schools from
(select ZIPCODE, count(*) as number_of_private_schools
from edge_1718_modeled.private_school_Beam_DF
group by ZIPCODE) pri join (select ZIPCODE, count(*) as number_of_public_schools
from edge_1718_modeled.public_school_Beam_DF
group by ZIPCODE) pub on pri.ZIPCODE = pub.ZIPCODE) combine on z.ZIPCODE = combine.ZIPCODE
order by number_of_private_schools desc
limit 20

,ZCTA5,Predominant_Class,number_of_private_schools,number_of_public_schools
0,33012,Lowest,22,9
1,33014,Lowest,17,8
2,33010,Lowest,16,7
3,33169,Middle,13,8
4,33030,Lowest,12,20
5,33311,Lowest,11,24
6,53209,Lowest,10,12
7,11691,Lowest,10,16
8,33013,Lowest,9,8
9,19144,Lowest,8,7


#### create view

In [10]:
%%bigquery
create or replace view reporting.v_School_with_Income_Class as
select i.ZCTA5, Predominant_Class, number_of_private_schools, number_of_public_schools
from (sashimi-266523.acs_2018_modeled.Income_Beam_DF i join sashimi-266523.uds_mapper_modeled.ZIP_To_ZCTA5_Beam_DF z on i.ZCTA5 = z.ZCTA5) join 
(select pri.ZIPCODE, number_of_private_schools, number_of_public_schools from
(select ZIPCODE, count(*) as number_of_private_schools
from sashimi-266523.edge_1718_modeled.private_school_Beam_DF
group by ZIPCODE) pri join (select ZIPCODE, count(*) as number_of_public_schools
from sashimi-266523.edge_1718_modeled.public_school_Beam_DF
group by ZIPCODE) pub on pri.ZIPCODE = pub.ZIPCODE) combine on z.ZIPCODE = combine.ZIPCODE
order by number_of_private_schools desc

""


### Number of postsecondary schools in comparison of private/public enrollments in postsecondary schools in the area/neighborhood

In [12]:
%%bigquery
select s.ZCTA5, (College_Undergrad_Public + Grad_HigherEdu_Public) as Public_enrollment, (College_Undergrad_Private + Grad_HigherEdu_Private) as Private_enrollment, total_number_of_schools
from (acs_2018_modeled.PrivatePublic_School_Enrollment_Beam_DF s join uds_mapper_modeled.ZIP_To_ZCTA5_Beam_DF z on s.ZCTA5 = z.ZCTA5) join 
(select ZIPCODE, count(*) as total_number_of_schools from
edge_1718_modeled.postsecondary_school_Beam_DF
group by ZIPCODE) p on z.ZIPCODE = p.ZIPCODE
order by total_number_of_schools desc
limit 10

,ZCTA5,Public_enrollment,Private_enrollment,total_number_of_schools
0,33012,3510,1223,11
1,02115,947,16361,11
2,00725,1688,4358,10
3,92101,2464,1446,10
4,91711,2975,4425,10
5,10011,1064,2215,10
6,85021,1768,532,10
7,08701,1416,6849,10
8,10001,842,1707,9
9,10952,644,1739,9


#### create view

In [13]:
%%bigquery
create or replace view reporting.v_Postsecondary_vs_PublicPrivate_enrollment as
select s.ZCTA5, (College_Undergrad_Public + Grad_HigherEdu_Public) as Public_enrollment,
(College_Undergrad_Private + Grad_HigherEdu_Private) as Private_enrollment, total_number_of_schools
from (sashimi-266523.acs_2018_modeled.PrivatePublic_School_Enrollment_Beam_DF s join sashimi-266523.uds_mapper_modeled.ZIP_To_ZCTA5_Beam_DF z on s.ZCTA5 = z.ZCTA5) join 
(select ZIPCODE, count(*) as total_number_of_schools from sashimi-266523.edge_1718_modeled.postsecondary_school_Beam_DF
group by ZIPCODE) p on z.ZIPCODE = p.ZIPCODE
order by total_number_of_schools desc

""
